# Rpi serial ports

* default:
   * hardware serial for BT
   * miniserial for console
   
* swap these:
   * [Official Documentation](https://www.raspberrypi.org/documentation/configuration/uart.md)
   * [Detailed Instructions](https://www.circuits.dk/setup-raspberry-pi-3-gpio-uart/)


## 1) Remap Serial Ports (BT uses miniuart, Pi header uses main uart)

Note: also fix Pi clock frequency for BT to work (havn't done that).

In [1]:
!tail /boot/config.txt

# core_freq_min=500
dtoverlay=miniuart-bt

[pi4]
# Enable DRM VC4 V3D driver on top of the dispmanx display stack
dtoverlay=vc4-fkms-v3d
max_framebuffers=2

[all]
#dtoverlay=vc4-fkms-v3d


In [4]:
!ls /dev/serial*

/dev/serial0  /dev/serial1


Pi 3, Pi Zero W: serial0->ttyS0 points to GPIO pins 8, 10.

So where possible refer to the serial port via it’s alias of “serial0” and your code should work on both Raspberry Pi 3 and other Raspberry Pi’s.

In [2]:
!ls -l /dev/serial* /dev/ttyS* /dev/ttyA*

lrwxrwxrwx 1 root root          7 Mar  3 10:02 /dev/serial0 -> ttyAMA0
lrwxrwxrwx 1 root root          5 Mar  3 10:02 /dev/serial1 -> ttyS0
crw-rw---- 1 root dialout 204, 64 Mar  3 10:02 /dev/ttyAMA0
crw-rw---- 1 root dialout   4, 64 Mar  3 10:02 /dev/ttyS0


## 2) Disable the Console

By default the Raspberry Pi uses the serial port for the “console” login and via a software service called “getty”.
If you are using the serial port for anything other than the console you need to disable it. This will be slightly different depending on whether you are running a Raspberry Pi 3 or not.
For non Raspberry Pi 3 machines, remember it’s /dev/ttyAMA0 that is linked to the getty (console) service.

For Raspberry Pi 3’s the command is referencing /dev/ttyS0:

In [3]:
!sudo systemctl stop serial-getty@ttyS0.service
!sudo systemctl disable serial-getty@ttyS0.service

In [4]:
# remove 'console=serial0,115200' from `/boot/cmdline.txt`

%cat  /boot/cmdline.txt

console=tty1 root=PARTUUID=4c969853-02 rootfstype=ext4 elevator=deadline fsck.repair=yes rootwait


In [ ]:
!sudo reboot now

## 3) Test

In [ ]:
import serial, time

with serial.Serial("/dev/serial0", baudrate=9600, timeout=3.0) as dev:
    while True:
        dev.write(b"Hello world!")
        rcv = dev.read(10)
        print(rcv)